In [ ]:
import py21cmcast as p21c
import numpy as np 

from astropy import units

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## I. Create the configuration files for varying parameters

See the examples of master configuration files in the config folder. Running **init\_fisher\_from\_fiducial** will create a folder (named after the run name set in the master configuration file) and sub configuration files inside (which correspond to the variation of each parameter). Once they are created each can by run with the script [**run\_fisher.py**](https://github.com/gaetanfacchinetti/21cmCAST/blob/main/scripts/exec/run_fisher.py). Note that the script takes at least one argument (the name of the sub configuration files to run) and three optional: the number of threads, the number of the first seed, and the number of times the same script should be run with increasing seeds. The latter is only usefull to run several fiducial cases in order to determine what the 'average Universe' looks like. In practice, for a Fisher forecast one should fix the seed to a single value for all the runs. 

In [ ]:
configuration_file = "/scratch/ulb/physth_fi/gfacchin/runs_21cmCAST/config/constraint_nobkr.config"
p21c.init_runs_from_fiducial(configuration_file, clean_existing_dir=False)

## II. Define the grid of modes and redshifts

Let us call the function **define\_grid\_modes\_redshits()**. It returns the bin edges for a fixed grid of modes and redshifts.

- The redshift bins are computed from the frequency according to the formula
\begin{equation}
z_n \equiv \frac{f_{21}}{\frac{f_{21}}{1+z_0} - n B} -1
\end{equation}
where $B$ is the bandwidth and $f_{21}$ is the 21cm frequency.

- The mode bins are computed from the bandwidth according to the formula
\begin{equation}
k_n \equiv {\rm max} \left\{k_{\rm min},  \delta k \right\} + n \delta k \quad {\rm with} \quad \delta k \equiv 2\pi \frac{f_{21}}{B} \frac{H(z_0)}{c(1+z_0)^2}
\end{equation}
This definition corresponds to the bins set by **21cmSense**. Note that the choice of redshift $z_0$ fixes the step of the mode bins.

In [ ]:
z_bins, z_array, k_bins = p21c.define_grid_modes_redshifts(6., 8 * units.MHz, z_max = 22, k_min = 0.1 / units.Mpc, k_max = 1 / units.Mpc)
print("The redshift bin edges are:", z_bins)
print("The mode bin edges are:", k_bins)

## III. Define the fiducial model and get its power spectrum

A **Fiducial** object can be defined by specifying the path where the lightcones are saved (depends on what was set on the config file) as well as  the redshifts and modes bin edges. Note that we must set by hand whether the mode bins are linearly of logarithmically spaced with the option **logk**  that is by default **False** as the output mode bin deges returned by **define\_grid\_modes\_redshits()** are linearly spaced. Further options can be specified, such as a fraction of modeling error **frac\_mod** (see below).

In [ ]:
fiducial = p21c.Fiducial("../runs/CONSTRAINT_BKR_MINIHALOS", z_bins, z_array, k_bins, False, load=True)

Once the fiducial model is defined, we can check some of its properties:
- by plotting the neutral hydrogen fraction 'xH_box' (provided they have been asked as output of the lightcone) and computing the optical depth to reionization
- by computing the reduced $\chi^2$ value from the UV luminosity data

In [ ]:
fiducial.plot_xH_box()
fiducial.plot_global_signal()

print('The optical depth to reionization is:', fiducial.tau_ion)

# Uncomment only if you are not using minihalos, not implemented otherwise
#chi2 = fiducial.chi2_UV_luminosity_functions(plot = True)
#print('The reduced chi^2 from UV luminosity data is:', chi2)

## IV. Evaluate experimental noise

The experimental noise is derived using **21cmSense**. To that end one simply has to pass an 'observation_set' attribute to the compute_sensitivity() method of the fiducial object. If none are passed the default is used `default_HERA`. To use a custom instrument create a list of 21cmSense Observations objects the length of 'fiducial.z_array' (each observation corresponding to the redshift in `fiducial.z_array`).

In [ ]:
%%capture 
## Remove the long 21cmSense output

fiducial.compute_sensitivity()
fiducial.plot_power_spectrum()

## V. Define the Parameter objects

(This part is the longest and can take minutes or hours depending on the box resolution). Here all power spectra and associated derivatives are pre-computed and stored as atrribute of the **Parameter** object. With the load and save options of the **Parameter** class, if this part of code has been executed once, then all the objects can be reloaded in an instant (since every pre-computed quantity is stored in .npz and .pkl files).

In [ ]:
## Classical set of parameters
#parameter_names = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC',  'M_TURN', 't_STAR', 'L_X', 'NU_X_THRESH', 'DM_DECAY_RATE']
parameter_names = ['F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI',  't_STAR', 
                   'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 'NU_X_THRESH', 'DM_DECAY_RATE']

params = [None] * len(parameter_names)

for iname, name in enumerate(parameter_names) :  
    params[iname] = p21c.Parameter(fiducial=fiducial, name=name, verbose = False, plot = True, load=True)
    params[iname].plot_power_spectra(color=['b', 'k', 'r'])

## VI. Evaluate the Fisher matrix and its inverse

From the list of parameters objects defined above one can directly compute the Fisher matrix with the **evaluate\_fisher\_matrix()** function. One can add a modeling noise to the power spectrum with the attribute **frac\_noise** of the fiducial (as a percentage of the fiducial value). The triangle plot figure can be produced with the function **make\_triangle\_plot()** and plotted. Finally, the matrices can be displayed nicely with the **display\_matrix()** function.

In [ ]:
fiducial.frac_noise = 0.2
fisher_matrix     = p21c.evaluate_fisher_matrix(params)
covariance_matrix = np.linalg.inv(fisher_matrix['matrix'])

In [ ]:
covariance_dict = {}

# Transform the covariance matrix into covariance dictionnaries that can be plotted

for i, name_i in enumerate(fisher_matrix['name']):
    covariance_dict[name_i] = {}
    for j, name_j in enumerate(fisher_matrix['name']):
        covariance_dict[name_i][name_j] = covariance_matrix[i, j]

fig = p21c.make_triangle_plot([covariance_dict], [fiducial.astro_params], color=['dodgerblue'], alpha=[1], 
                              params_to_plot = ['F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI',  't_STAR', 'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 'NU_X_THRESH', 'DM_DECAY_RATE'])

fig.savefig(fiducial.dir_path + '/triangle_plot.pdf')

In [ ]:
p21c.display_matrix(fisher_matrix['matrix'], fisher_matrix['name'])
print("-------")
p21c.display_matrix(covariance_matrix, fisher_matrix['name'])
print("-------")
p21c.display_matrix(np.sqrt(np.abs(covariance_matrix)), fisher_matrix['name'])

In [ ]:
#l10_mass_str_arr = ['2.0', '2.5', '3.0', '3.5', '4.0', '4.5', '5.0', '5.5', '6.0', '6.5', '7.0', '7.5', '8.0', '8.5', '9.0', '9.5', '10.0', '10.5', '11.0', '11.5', '12.0']
l10_mass_str_arr = ['6.25', '6.5', '6.75', '7.0', '7.25', '7.5', '7.75', '8.0', '8.25', '8.5', '8.75', '9.0', '9.25', '9.5', '9.75', '10.0', '10.25','10.5', '10.75', '11.0', '11.25', '11.5', '11.75', '12.0']
l10_mass_arr = [np.float(l10_mass_str) for l10_mass_str in  l10_mass_str_arr]
gamma_array  = [1e-36, 1e-35, 1e-34, 1e-33, 1e-32, 1e-31, 1e-30, 1e-29, 1e-28, 1e-27, 1e-26, 1e-25, 1e-24, 1e-23]

for im, l10_mass in enumerate(l10_mass_arr):
    primary = 'elec_delta' if l10_mass <= np.log10(8e+9) else 'e'
    p21c.make_config_one_varying_param("/scratch/ulb/physth_fi/gfacchin/runs_21cmCAST/config/constraint_nobkr.config", 'DM_DECAY_RATE', gamma_array , mod_astro_params = {'DM_LOG10_MASS' : l10_mass},  mod_flag_options = {'DM_PRIMARY' : primary}, add_file_name= 'm' + l10_mass_str_arr[im])